<a href="https://colab.research.google.com/github/BrockDSL/ARCH_Data_Explore/blob/main/Prep_Muni_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing of Muni data

In [ ]:
import pandas as pd
import gdown

#Sentiment just for fun, going to use VADER
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')from google.colab import files


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', False)
pd.set_option('display.max_rows', 200)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
#Need to update to Dec 31 Crawl infoy
gdown.download('https://drive.google.com/u/0/uc?id=1SzIpVc2fYLQTIasZZi3FnR7WhRRhsbJz&export=download','web-pages_full.csv.gz',quiet=False)
!gunzip -k web-pages_full.csv.gz

Downloading...
From: https://drive.google.com/u/0/uc?id=1SzIpVc2fYLQTIasZZi3FnR7WhRRhsbJz&export=download
To: /content/web-pages_full.csv.gz
100%|██████████| 602M/602M [00:07<00:00, 81.4MB/s]


In [ ]:
archive_text = pd.read_csv("web-pages_full.csv")

In [ ]:
archive_text['crawl_date']= pd.to_datetime(archive_text['crawl_date'],format='%Y%m%d')
archive_text = archive_text[archive_text['language'] =='en']

In [ ]:
archive_text.groupby(["domain"]).count().sort_values(by="crawl_date", ascending=False)[0:50]

,crawl_date,url,mime_type_web_server,mime_type_tika,language,content
domain,,,,,,
brocku.ca,214524,214524,214524,214524,214524,214524
youtube.com,44443,44443,44443,44443,44443,44443
gncc.ca,44428,44428,44428,44428,44428,44428
twitter.com,36782,36782,36782,36782,36782,36782
portcolborne.ca,26079,26079,26079,26079,26079,26079
gobadgers.ca,21022,21022,21022,21022,21022,21022
niagaraanglican.ca,18808,18808,18808,18808,18808,18808
niagarahealth.on.ca,15179,15179,15179,15179,15179,15179
pelhamlibrary.on.ca,14221,14221,14221,14221,14221,14221


In [ ]:
#Municiple domains into a specific dataframe

municiple_domains = [

  "niagararegion.ca",
  "notl.com",
  "stcatharines.ca",
  "niagarafalls.ca",
  "lincoln.ca",
  "forterie.ca",
  "grimsby.ca",
  "portcolborne.ca",
  "thorold.ca",
  "welland.ca",
  "westlincoln.ca",
  "wainfleet.ca",
  "pelham.ca"
]

muni_text = archive_text[archive_text.domain.isin(municiple_domains)]
muni_text = muni_text.reset_index()

In [ ]:
del(muni_text['length'])

In [ ]:
#length via itertuples, no index

lengths = []
for row in muni_text.itertuples(index=False):
  lengths.append(int(len(row.content)))

len_df = pd.DataFrame(lengths,columns=["length"])

muni_text = muni_text.join(len_df)

In [ ]:
#VADER
v_pos = []
v_neg = []
v_neu = []
v_comp = []

sid = SentimentIntensityAnalyzer()
for row in muni_text.itertuples(index=False):
    ss = sid.polarity_scores(row.content)
    v_pos.append(float(ss["pos"]))
    v_neg.append(float(ss["neg"]))
    v_neu.append(float(ss["neu"]))
    v_comp.append(float(ss["compound"]))



In [ ]:
v_pos_df = pd.DataFrame(v_pos,columns=["v_pos"])
v_neg_df = pd.DataFrame(v_neg,columns=["v_neg"])
v_neu_df = pd.DataFrame(v_neu,columns=["v_neu"])
v_comp_df = pd.DataFrame(v_comp,columns=["v_comp"])

muni_text = muni_text.join(v_pos_df)
muni_text = muni_text.join(v_neg_df)
muni_text = muni_text.join(v_neu_df)
muni_text = muni_text.join(v_comp_df)

In [ ]:
#just to make sure
muni_text.sample(2)

In [ ]:
#write of final CSV and download

muni_text.to_csv("municipal_text.csv",index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -lh

total 5.2G
-rw-r--r-- 1 root root 467M Jan  6 17:36 municipal_text.csv
drwxr-xr-x 1 root root 4.0K Dec 23 14:32 sample_data
-rw-r--r-- 1 root root 4.1G Jan  6 15:55 web-pages_full.csv
-rw-r--r-- 1 root root 575M Jan  6 15:55 web-pages_full.csv.gz


In [ ]:
files.download("municipal_text.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>